<a href="https://colab.research.google.com/github/janvierk/MRP/blob/main/MRP_mBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/MRP')

In [ ]:
#installing requitred libries and supressing  the print-out
#!pip install requests==2.31.0 --upgrade -q > NUL
!pip install accelerate -U -q > NUL
!pip install transformers -q > NUL
!pip install torch -q > NUL
#!pip install -U transformers -q >NULip install datasets -q > NUL
!pip install tensorflow -q > NUL
!pip install datasets -q > NUL
!pip install evaluate -q > NUL

In [ ]:
#Required packages and libries
#Required packages and libries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers  import Trainer
from transformers import TrainingArguments
from transformers import XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification
from transformers import MT5ForConditionalGeneration
from transformers import T5Tokenizer
import seaborn as sn
import matplotlib
from transformers import AutoTokenizer, AutoModelForTokenClassification
import transformers as tf
import torch
from datasets import load_dataset
import datasets
import  evaluate

In [ ]:
#importing the clean dataset to train the model
swahili_dat = pd.read_csv("swahili_clean.csv")

In [ ]:
english_dat = pd.read_csv("english_news_clean")

In [ ]:
english_dat.shape[0]

29544

In [ ]:
swahili_dat.shape[0]

29544

In [ ]:
english_dat.shape[0]

29544

In [ ]:
#shuffling the datasets
swahili_dat= swahili_dat.sample(frac =1, random_state =42)
english_dat = english_dat.sample(frac =1, random_state = 42)

In [ ]:
english_dat.value_counts("label")

,count
label,
0,16000
1,11720
2,854
3,557
4,413


In [ ]:

#creating a small swahili dataset

In [ ]:
#dividing swahili dataset for training, development and testing
from sklearn.model_selection import train_test_split
swahili_train, swahili_development = train_test_split(swahili_dat, test_size=0.4,  random_state = 42 )
swahili_validation, swahili_testing = train_test_split(swahili_development, test_size = 0.5, random_state = 42)

In [ ]:
#resetting the index
swahili_testing.reset_index(drop = True, inplace = True)
swahili_validation.reset_index(drop = True, inplace = True)
swahili_train.reset_index(drop = True, inplace = True)

In [ ]:
#saving the dataset
swahili_testing.to_csv("swahili_testing.csv", index=False)
swahili_train.to_csv("swahili_train.csv", index =False)
swahili_validation.to_csv("swahili_validation.csv", index = False)

In [ ]:
#dividing cnn dataset for training, validation and testing
english_train, english_development = train_test_split(english_dat, test_size=0.4,  random_state = 42 )
english_validation, english_testing = train_test_split(english_development, test_size = 0.5, random_state = 42)

In [ ]:
#resetting the index
english_testing.reset_index(drop = True, inplace = True)
english_validation.reset_index(drop = True, inplace = True)
english_train.reset_index(drop = True, inplace = True)

In [ ]:
#saving the english dataset
english_testing.to_csv("english_testing.csv", index=False)
english_train.to_csv("english_train.csv", index =False)
english_validation.to_csv("english_validation.csv", index = False)

In [ ]:
#Uisng Bert tokenizer for tokernization of the data
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_data(text):
  return tokenizer(text,
                   padding = "max_length",
                   truncation = True,
  )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
#tokenizing swahili data
tokenized_swahili_train =  swahili_train["text"].apply(tokenize_data)
tokenized_swahili_validation = swahili_validation["text"].apply(tokenize_data)
tokenized_swahili_testing = swahili_testing["text"].apply(tokenize_data)

In [ ]:
#tokenizing english data
tokenized_english_train =  english_train["text"].apply(tokenize_data)
tokenized_english_validation = english_validation["text"].apply(tokenize_data)
tokenized_english_testing = english_testing["text"].apply(tokenize_data)

In [ ]:
#checking if the text has been tokenized accordingly
for i in range(3):
    print(f"Tokenized entry {i+1}:")
    print(f"Input IDs: {tokenized_swahili_train.iloc[i]['input_ids']}")
    print(f"Token Type IDs: {tokenized_swahili_train.iloc[i]['token_type_ids']}")
    print(f"Attention Mask: {tokenized_swahili_train.iloc[i]['attention_mask']}")
    print("\n")

Tokenized entry 1:
Input IDs: [101, 28941, 18845, 10116, 18939, 103695, 10537, 10879, 69553, 26127, 10113, 14876, 10465, 19502, 10133, 15176, 12589, 10879, 69553, 13024, 10381, 10113, 10879, 35481, 37861, 10113, 99946, 10401, 107591, 20338, 10132, 10883, 16423, 10908, 28532, 52866, 95342, 10549, 106306, 11044, 10109, 44795, 15914, 15529, 10133, 13171, 10559, 102161, 11471, 69553, 11952, 76372, 13723, 58671, 11471, 21511, 14590, 11037, 39796, 10549, 11856, 14383, 64470, 13605, 10133, 33376, 10219, 10132, 106712, 10550, 10133, 10549, 10911, 12806, 10549, 23605, 10116, 10549, 10148, 10246, 10549, 123, 118, 123, 28532, 158, 14394, 10320, 11471, 25633, 13369, 117, 35368, 119, 13026, 28167, 11471, 12865, 49581, 11460, 10228, 10133, 117, 11118, 10820, 10392, 13565, 10369, 117, 11471, 41947, 78314, 19830, 82125, 10113, 14876, 18089, 15694, 23661, 86137, 13133, 11282, 13605, 10133, 14266, 15694, 37583, 12177, 10113, 189, 81111, 10775, 11471, 10133, 11471, 75459, 10376, 11037, 11471, 32874, 4986

In [ ]:
#extracting input ids and attention mask for swahili training
input_ids_swahili_train= [item["input_ids"] for item in tokenized_swahili_train]
attention_mask_swahili_train = [item["attention_mask"] for item in tokenized_swahili_train]
swahili_encodings_train = {"input_ids": input_ids_swahili_train,
             "attention_mask": attention_mask_swahili_train}

In [ ]:
#extracting input ids and attention mask for swahili validation
input_ids_swahili_validation= [item["input_ids"] for item in tokenized_swahili_validation]
attention_mask_swahili_validation = [item["attention_mask"] for item in tokenized_swahili_validation]
swahili_encodings_validation = {"input_ids": input_ids_swahili_validation,
             "attention_mask": attention_mask_swahili_validation}

In [ ]:
#extracting input ids and attention mask for swahili testing
input_ids_swahili_testing= [item["input_ids"] for item in tokenized_swahili_testing]
attention_mask_swahili_testing = [item["attention_mask"] for item in tokenized_swahili_testing]
swahili_encodings_testing = {"input_ids": input_ids_swahili_testing,
             "attention_mask": attention_mask_swahili_testing}

In [ ]:
#extracting input ids and attention mask for english training
input_ids_english_train= [item["input_ids"] for item in tokenized_english_train]
attention_mask_english_train = [item["attention_mask"] for item in tokenized_english_train]
english_encodings_train = {"input_ids": input_ids_english_train,
             "attention_mask": attention_mask_english_train}

In [ ]:
#extracting input ids and attention mask for english validation
input_ids_english_validation= [item["input_ids"] for item in tokenized_english_validation]
attention_mask_english_validation = [item["attention_mask"] for item in tokenized_english_validation]
english_encodings_validation = {"input_ids": input_ids_english_validation,
             "attention_mask": attention_mask_english_validation}

In [ ]:
#extracting input ids and attention mask for english testing
input_ids_english_testing= [item["input_ids"] for item in tokenized_english_testing]
attention_mask_english_testing = [item["attention_mask"] for item in tokenized_english_testing]
english_encodings_testing = {"input_ids": input_ids_english_testing,
             "attention_mask": attention_mask_english_testing}

In [ ]:
#getting labels for swahili datasets
swahili_train_labels = swahili_train["label"]
swahili_validation_labels = swahili_validation["label"]
swahili_testing_labels = swahili_testing["label"]

In [ ]:
#getting labels for english datasets
english_train_labels = english_train["label"]
english_validation_labels = english_validation["label"]
english_testing_labels = english_testing["label"]

In [ ]:
#creating tensor data to feed into the model
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset (Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

In [ ]:
#swahili ready data
#english ready data
swahili_ready_train = TextDataset(swahili_encodings_train, swahili_train_labels)
swahili_ready_validation = TextDataset(swahili_encodings_validation, swahili_validation_labels)
swahili_ready_testing = TextDataset(swahili_encodings_testing, swahili_testing_labels)

In [ ]:
#english ready data
english_ready_train = TextDataset(english_encodings_train, english_train_labels)
english_ready_validation = TextDataset(english_encodings_validation, english_validation_labels)
english_ready_testing = TextDataset(english_encodings_testing, english_testing_labels)

In [ ]:
def inspect_dataset(dataset, num_samples):
  for i in range(min(num_samples, len(dataset))):
    item = dataset[i]
    print(f"samples{i}:")
    for key, value in item.items():
      print(f" {key} : {value}")
    print("\n")


In [ ]:
#Building  a simple mBert model for small swahili dataset
!pip install -U accelerate -q >NUL
!pip install -U transformers -q >NUL
swahili_mbert = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                                  num_labels = swahili_train["label"].nunique())
english_mbert =  BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                                  num_labels = english_train["label"].nunique())

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#training arguments
training_args_sm = TrainingArguments(
    output_dir="./sm_results1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./sm_logging1",
    logging_steps=20,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch"
)
training_args_em = TrainingArguments(
    output_dir="./em_results1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./em_logging1",
    logging_steps=20,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch"
)

In [ ]:
#computing the evaluation metrics
import evaluate
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")


In [ ]:
from transformers import EarlyStoppingCallback
from sklearn.metrics import confusion_matrix

In [ ]:
"""#computing the evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }"""

In [ ]:
#computing the evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')
    conf_matrix = confusion_matrix(labels, preds)
    conf_matrix_list = conf_matrix.tolist()

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall'],
        'conf_matrix': conf_matrix_list
    }

In [ ]:
import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.nn as nn
# Calculate class weights using sklearn's compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(swahili_train_labels), y=swahili_train_labels)

# Convert class weights to a tensor for PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
#Define your custom Trainer class with class weights
class CustomTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.args.device)  # Move weights to the appropriate device

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Define the loss function with class weights
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
mbert_trainer_swahili = CustomTrainer(
    model=swahili_mbert,
    args=training_args_sm,
    train_dataset=swahili_ready_train,
    eval_dataset=swahili_ready_validation,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    class_weights=class_weights)  # Pass class weights to the custom trainer

In [ ]:
"""mbert_trainer_swahili= Trainer(
    model=swahili_mbert,
    args=training_args_sm,
    train_dataset=swahili_ready_train,
    eval_dataset=swahili_ready_validation,
    compute_metrics=compute_metrics,  # Use the modified compute_metrics function
)"""

In [ ]:
"""mbert_trainer_english = Trainer(
    model=english_mbert,
    args=training_args_em,
    train_dataset=english_ready_train,
    eval_dataset=english_ready_validation,
    compute_metrics=compute_metrics,  # Use the modified compute_metrics function
)"""

In [ ]:
mbert_trainer_english = CustomTrainer(
    model=english_mbert,
    args=training_args_em,
    train_dataset=english_ready_train,
    eval_dataset=english_ready_validation,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    class_weights=class_weights)  # Pass class weights to the custom trainer

In [ ]:
mbert_trainer_swahili.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Conf Matrix
1,0.483900,0.563862,0.912845,0.909076,0.909021,0.912845,"[[3031, 19, 55, 36, 74], [95, 240, 0, 0, 3], [25, 1, 501, 3, 22], [114, 0, 3, 79, 4], [26, 14, 21, 0, 1543]]"
2,0.524100,0.490852,0.920460,0.920211,0.921332,0.920460,"[[2998, 15, 49, 88, 65], [85, 248, 0, 1, 4], [14, 0, 514, 4, 20], [71, 0, 7, 122, 0], [26, 9, 12, 0, 1557]]"
3,0.281800,0.488230,0.905568,0.906926,0.910563,0.905568,"[[2871, 126, 50, 88, 80], [51, 282, 0, 1, 4], [11, 0, 512, 4, 25], [71, 0, 8, 120, 1], [18, 7, 12, 1, 1566]]"


Trainer is attempting to log a value of "[[3031, 19, 55, 36, 74], [95, 240, 0, 0, 3], [25, 1, 501, 3, 22], [114, 0, 3, 79, 4], [26, 14, 21, 0, 1543]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[2998, 15, 49, 88, 65], [85, 248, 0, 1, 4], [14, 0, 514, 4, 20], [71, 0, 7, 122, 0], [26, 9, 12, 0, 1557]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[2871, 126, 50, 88, 80], [51, 282, 0, 1, 4], [11, 0, 512, 4, 25], [71, 0, 8, 120, 1], [18, 7, 12, 1, 1566]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=3324, training_loss=0.5303077184192827, metrics={'train_runtime': 1317.6837, 'train_samples_per_second': 40.357, 'train_steps_per_second': 2.523, 'total_flos': 1.3992096579950592e+16, 'train_loss': 0.5303077184192827, 'epoch': 3.0})

In [ ]:
mbert_trainer_english.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.144800,0.149075,0.965307,0.962905,0.963985,0.965307
2,0.138900,0.122514,0.968184,0.968136,0.968153,0.968184
3,0.180300,0.146749,0.967507,0.967556,0.967622,0.967507


TrainOutput(global_step=3324, training_loss=0.16962960517094453, metrics={'train_runtime': 2611.643, 'train_samples_per_second': 20.362, 'train_steps_per_second': 1.273, 'total_flos': 1.3992096579950592e+16, 'train_loss': 0.16962960517094453, 'epoch': 3.0})

In [ ]:
#testing mbert swahili model on testing dataset
mbert_test_swahili = mbert_trainer_swahili.evaluate(eval_dataset=swahili_ready_testing)

In [ ]:
#mbert_trainer_swahili.evaluate(eval_dataset=swahili_ready_testing)

In [ ]:
mbert_test_english = mbert_trainer_english.evaluate(eval_dataset=english_ready_testing)

In [ ]:
#accessing the logging  history for visualization
training_logs_swahili = mbert_trainer_swahili.state.log_history
training_logs_english = mbert_trainer_english.state.log_history

In [ ]:
print(training_logs_swahili)

[{'loss': 1.5642, 'grad_norm': 29.795320510864258, 'learning_rate': 5e-06, 'epoch': 0.018050541516245487, 'step': 20}, {'loss': 1.3504, 'grad_norm': 5.276247978210449, 'learning_rate': 1e-05, 'epoch': 0.036101083032490974, 'step': 40}, {'loss': 1.018, 'grad_norm': 9.203198432922363, 'learning_rate': 1.5e-05, 'epoch': 0.05415162454873646, 'step': 60}, {'loss': 0.7667, 'grad_norm': 9.459148406982422, 'learning_rate': 2e-05, 'epoch': 0.07220216606498195, 'step': 80}, {'loss': 0.556, 'grad_norm': 3.1963772773742676, 'learning_rate': 2.5e-05, 'epoch': 0.09025270758122744, 'step': 100}, {'loss': 0.5624, 'grad_norm': 44.14239501953125, 'learning_rate': 3e-05, 'epoch': 0.10830324909747292, 'step': 120}, {'loss': 0.6394, 'grad_norm': 6.092493534088135, 'learning_rate': 3.5e-05, 'epoch': 0.1263537906137184, 'step': 140}, {'loss': 0.6771, 'grad_norm': 18.806947708129883, 'learning_rate': 4e-05, 'epoch': 0.1444043321299639, 'step': 160}, {'loss': 0.4916, 'grad_norm': 8.452995300292969, 'learning_r

In [ ]:
print(training_logs_english)

[{'loss': 1.5021, 'grad_norm': 7.758094787597656, 'learning_rate': 5e-06, 'epoch': 0.018050541516245487, 'step': 20}, {'loss': 1.176, 'grad_norm': 8.735230445861816, 'learning_rate': 1e-05, 'epoch': 0.036101083032490974, 'step': 40}, {'loss': 0.6972, 'grad_norm': 6.7875471115112305, 'learning_rate': 1.5e-05, 'epoch': 0.05415162454873646, 'step': 60}, {'loss': 0.464, 'grad_norm': 2.4097936153411865, 'learning_rate': 2e-05, 'epoch': 0.07220216606498195, 'step': 80}, {'loss': 0.4102, 'grad_norm': 1.8956880569458008, 'learning_rate': 2.5e-05, 'epoch': 0.09025270758122744, 'step': 100}, {'loss': 0.4142, 'grad_norm': 3.624096155166626, 'learning_rate': 3e-05, 'epoch': 0.10830324909747292, 'step': 120}, {'loss': 0.3122, 'grad_norm': 2.628129720687866, 'learning_rate': 3.5e-05, 'epoch': 0.1263537906137184, 'step': 140}, {'loss': 0.2911, 'grad_norm': 7.894004821777344, 'learning_rate': 4e-05, 'epoch': 0.1444043321299639, 'step': 160}, {'loss': 0.3599, 'grad_norm': 25.145278930664062, 'learning_

In [ ]:
#converting into pandas dataframe
df_logs_swahili = pd.DataFrame(training_logs_swahili)
df_logs_english = pd.DataFrame(training_logs_english)

In [ ]:
print(df_logs_swahili.columns)
print(df_logs_english.columns)

Index(['loss', 'grad_norm', 'learning_rate', 'epoch', 'step', 'eval_loss',
       'eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall',
       'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second',
       'train_runtime', 'train_samples_per_second', 'train_steps_per_second',
       'total_flos', 'train_loss'],
      dtype='object')
Index(['loss', 'grad_norm', 'learning_rate', 'epoch', 'step', 'eval_loss',
       'eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall',
       'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second',
       'train_runtime', 'train_samples_per_second', 'train_steps_per_second',
       'total_flos', 'train_loss'],
      dtype='object')


In [ ]:
df_logs_swahili = df_logs_swahili[['epoch', 'eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall', 'eval_loss']]#.dropna()
df_logs_english = df_logs_english[['epoch', 'eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall', 'eval_loss']]#.dropna()

In [ ]:
df_logs_swahili.head()

,epoch,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_loss
55,1.0,0.883398,0.878364,0.877664,0.883398,0.390876
111,2.0,0.906752,0.900112,0.903388,0.906752,0.323488
168,3.0,0.919445,0.914459,0.915941,0.919445,0.292983
170,3.0,0.922322,0.917569,0.918431,0.922322,0.272779


In [ ]:
print(mbert_test_swahili)

{'eval_loss': 0.27277910709381104, 'eval_accuracy': 0.9223218818751058, 'eval_f1': 0.9175693652544786, 'eval_precision': 0.9184308592208992, 'eval_recall': 0.9223218818751058, 'eval_runtime': 86.7175, 'eval_samples_per_second': 68.141, 'eval_steps_per_second': 4.267, 'epoch': 3.0}


ValueError: If using all scalar values, you must pass an index

In [ ]:
epochs = []
train_losses = []
eval_losses = []

for log in logs:
    if 'epoch' in log:
        epochs.append(log['epoch'])
        if 'loss' in log:
            train_losses.append(log['loss'])
        if 'eval_loss' in log:
            eval_losses.append(log['eval_loss'])

# Plot the training loss and validation loss against epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs[:len(train_losses)], train_losses, marker='o', linestyle='-', color='r', label='Training Loss')
plt.plot(epochs[:len(eval_losses)], eval_losses, marker='o', linestyle='-', color='b', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss vs. Epoch')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'logs' is not defined